# Numerical Precision and Error

We review how numbers and characters are represented in memory, types of numerical error, and floating point number representation and arithmetic.  

## Computer Memory

Inside Memory, there exists millions/billions of switches.  Each switch has two states:

"On" = 1
"Off" = 0

This dual nature gives rise to "binary" - a base 2 sytstem.  One binary digit is a bit.

Traditionally, a group of 8 bits is a byte:

1 byte:  2$^8$  = 256 possible values <br\>
2 bytes: 2$^{16}$ = 65,536 possible values <br\>
4 bytes: 2$^{32}$ = 4,294,967,296 possible values <br\>
8 bytes: 2$^{64}$ = 1.844674407 x 10$^{19}$ possible values <br\>

Generally, these values are split between positive and negative values.  For example, 1 byte has a range of -128 to 127.

The only thing that a computer can store is bytes.

To store anything in a computer, you must first encode it, i.e. convert it to bytes. For example:

    If you want to store music, you must first encode it using MP3, WAV, etc.
    If you want to store a picture, you must first encode it using PNG, JPEG, etc.
    If you want to store text, you must first encode it using ASCII, UTF-8, etc.

So what about the basic types of variables we use?

### Strings

It depends on the encodings.  For example, try:

#help(len)
#help(str.encode)
print len('s'.encode("ASCII"))
print len('string'.encode("ASCII"))
print len('I am a string'.encode("ASCII"))
print len('I am a string'.encode("utf32"))

chineseString="你好"
print chineseString
print len(chineseString)

As you can see, the size of the string depends on 
 
 a) The number of characters in the string <br\>
 b) The encoding used for the string
 
The more bytes the encoding uses, the more characters that can be represented.  Here is a list of [Python standard encodings](https://docs.python.org/2.4/lib/standard-encodings.html).

### Integers

It depends on the size of the int!  Python automatically adds bytes to represent the integer given. Try this with the code below:

In [65]:
from math import ceil
anInt=1
print 'The number of bytes is {:d} for the integer {:d}.'.format(int(ceil(anInt.bit_length()/8.)), anInt)


The number of bytes is 1 for the number 1.


### Floats

Floats default to double precision on your system's architectures.  You can check the attributes of a float on your stystem by running:

In [92]:
from sys import float_info
print float_info.max
print float_info.min

print float_info.epsilon
print float_info.dig

1.79769313486e+308
2.22507385851e-308
2.22044604925e-16
15


But what if you want to control the size of your variables?  Python gives you a way to do that too!

In [89]:
import numpy as np
print 'Float Type  Bits in Exp  Bits in Mantissa  Total Bytes'
print '------------------------------------------------------'
for f in (np.float16, np.float32, np.float64, float):
    finfo = np.finfo(f)
    print '{} {:10d} {:14d} {:14d}'.format(finfo.dtype, finfo.nexp, finfo.nmant, (finfo.nexp+finfo.nmant+1)/8)

Float Type  Bits in Exp  Bits in Mantissa  Total Bytes
------------------------------------------------------
float16          5             10              2
float32          8             23              4
float64         11             52              8
float64         11             52              8


## Types of Error

There are three main types of numerical error:

  1) Roundoff error
  2) Range error
  3) Truncation error
  
Let's look at some examples of each:

### Roundoff Error
Roundoff error occurs because of the computing device's inability to deal with certain numbers. Such numbers need to be rounded off to some near approximation which is dependent on the word size used to represent numbers of the device.

#### Subtraction of two very different numbers:

In [118]:
x = 2.15E12
y = .0000000000000000125E12

print 'Python calculated {:.15e}'.format(x-y)
print 'The answer is {}'.format('2.1499999999999999875E12')

Python calculated 2.150000000000000e+12
The answer is 2.1499999999999999875E12
The difference between the known value and the calculated value is 0.000000000000000e+00


Did you get what you expected? Why?

Rather than using all these digits, floating-point hardware normally operates on a fixed number of digits. Suppose that the number of digits kept is p, and that when the smaller operand is shifted right, digits are simply discarded (as opposed to rounding). 

Consider another case:

In [116]:
x = 1.01E1
y = 0.993E1
z = x-y
print 'Using scientific notation {:.18e}'.format(z)

a = 10.1
b = 9.93
c = a-b
print 'Without scientific notation {:.18f}'.format(c)

print 'The difference between the known value and the calculated value is {:.18e}'.format(0.17-c)

Using scientific notation 1.699999999999999289e-01
Without scientific notation 0.169999999999999929
The difference between the known value and the calculated value is 8.326672684688674053e-17


#### Subtraction of two nearly equal numbers

Now what happens if we subtract two nearly equal numbers?  Try creating your own code below:

To understand a bit of what is under the hood, a notebook is just a single (JSON) file:

## Range Error

Where roundoff error deals with the appromation in the mantissa, range error is associated with the exponent.

Consider the Bohr radius:

$a_0 = \frac{4\pi \epsilon_0 \hbar^2}{m_e e^2} \approx 5.3E-11$

This is easily within the range of all floats.  However, consider the break down of the components:

$4\pi \epsilon_0 \hbar^2 \approx 1.24E-78$ <br\>
$m_e e^2 \approx 2.34E-68$

Does this matter?  Try the following code:

In [126]:
import numpy as np
num = np.float16(1.24E-78)
print 'Numerator = {:.15e}'.format(num)
denom = np.float16(2.34E-68)
print 'Denominator = {:.15e}'.format(denom)
print 'The Bohr radius = {:.15e}'.format(num/denom)

Numerator = 0.000000000000000e+00
Denominator = 0.000000000000000e+00
The Bohr radius = nan


C:\Users\James\Anaconda2\lib\site-packages\ipykernel_launcher.py:6: RuntimeWarning: invalid value encountered in half_scalars
  


What happened?  How could you fix it?  Do so in the code section below:

### Truncation Error

Truncation error refers to the error in a method, which occurs because some series (finite or infinite) is truncated to a fewer number of terms. Such errors are essentially algorithmic errors and we can predict the extent of the error that will occur in the method.

Consider the factorial operator:

$n! = n(n-1)(n-2)....(n-(n-1))$

What is a straightfoward way to implement this?  Do so below:

Another way to implement this is the Stirling Approximation:

$n! = \sqrt{2n\pi} \ n^n e^n(1+\frac{1}{12n}+\frac{1}{288n^2}+...)$

Implement the Stirling Approximation of the first order ($\mathcal{O} \left(\frac{1}{n} \right)$), second order($\mathcal{O} \left(\frac{1}{n^2} \right)$), and third order($\mathcal{O} \left(\frac{1}{n^3} \right)$).

What do you see regarding the trunctation error? What sign would you expect the next term to have?

## A Final Note

Not all error is equally bad.  If an error stays at one point in an algorithm and doesn't aggregate further as the calculation continues, then it's considered a numerically stable error. This happens when the error causes only a very small variation in the formula result. If the opposite occurs and the error propagates bigger as the calculation continues, then it is considered numerically unstable.  

Numerically stable errors are unavoidable for most practicle applications.

Numerically unstable errors are tragic...